In [ ]:
# Import dependencies.
import matplotlib
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import pandas as pd

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, text

In [ ]:
engine = create_engine("sqlite:///open_university.sqlite", echo=False)

## Load the data

The plan is to just look at the interaction with the course material to see if the student will be able to predict if the student will pass or fail.

Not including information about the background of the student (demographics) or the scores of the student assessments but include if they submit the assessments or not.

We are going to filter out students that withdraw from the course before the start of the course.

In [ ]:
student_ds = engine.execute(text("""
SELECT sI.code_module, sI.code_presentation, sI.id_student, sI.final_result, sR.date_registration, sR.date_unregistration
FROM studentInfo as sI
LEFT JOIN studentRegistration as sR ON sI.id_student = sR.id_student
WHERE NOT (sR.date_unregistration <= -11 AND sI.final_result = 'Withdrawn')
""")).fetchall()
student_df = pd.DataFrame(student_ds, columns=['code_module', 'code_presentation', 'id_student', 'final_result', 'date_registration', 'date_unregistration'])
student_df = student_df.astype({'code_module':'string', 'code_presentation':'string', 'id_student':'string', 'final_result':'string'})

In [ ]:
display(student_df)
display(student_df.describe())
display(student_df.nunique())
display(student_df['final_result'].value_counts())
student_df.info()

* I removed students that Withdrew before 11 days before the presentation starts because they whould not have participated in the course.
* It looks like out of the 27,295 students there were 36,388 registrations which means some students where registered more than once. Of all the registrations there where 11,592 unregistrations.
* 


In [ ]:
bf_course_df = student_df.loc[student_df['date_unregistration'] <= -11].copy()
display(bf_course_df.head())
display(bf_course_df.describe())
#display(student_df.loc[student_df['date_registration'].isnull()])
bf_course_df.info()

In [ ]:
## need to add the Vle connected to Student Vle
vle_ds = engine.execute(text("""
SELECT cast(vle.id_site as text), vle.code_module, vle.code_presentation, vle.activity_type, vle.week_from, vle.week_to, cast(sVle.id_student as text), sVle.date, SUM(sVLe.sum_click) as sum_click
FROM vle
LEFT JOIN studentVle AS sVle ON vle.id_site = sVle.id_site AND vle.code_presentation = sVle.code_presentation AND vle.code_module = sVle.code_module
GROUP BY vle.code_module, vle.code_presentation, sVle.id_student
""")).fetchall()
vle_df = pd.DataFrame(vle_ds, columns=['id_site', 'code_module', 'code_presentation', 'activity_type', 'week_from', 'week_to', 'id_student', 'date', 'sum_click'])
vle_df = vle_df.astype({'id_site':'string', 'code_module':'string', 'code_presentation':'string', 'activity_type':'string', 'id_student':'string'})

In [ ]:
display(vle_df)
display(vle_df.nunique())
vle_df.info()

In [ ]:
ass_ds = engine.execute(text("""
SELECT cast(sAss.id_student as text), ass.code_module, ass.code_presentation, cast(sAss.id_assessment as text), sAss.date_submitted, ass.date
FROM studentAssessment as sAss
LEFT JOIN assessments as ass ON sAss.id_assessment = ass.id_assessment
""")).fetchall()

assessment_df = pd.DataFrame(ass_ds, columns=['id_student', 'code_module', 'code_presentation', 'id_assessent', 'date_submitted', 'date'])
display(assessment_df.describe())
assessment_df.info()

In [ ]:
students_assessed = list(assessment_df['id_student'].unique())
all_students = list(student_df['id_student'].unique())
print("""
We are looking at the students that haven't haded in any assessments.
* We don't want them to skew our data as they never participated.
* There could be reasons but for the momst part the students either withdrew or failed.
* over 75% had unregistered by the 27th day of the presetation.
* It could only be assumed that the 2 that passed had prior learning or some other reason.
""")
display(f"Number of students assessed/total students: {len(list(students_assessed))}/{len(list(all_students))}")

student_not_assessed_df = student_df[~student_df['id_student'].isin(students_assessed)].copy()
display(student_not_assessed_df.nunique())
display(student_not_assessed_df['final_result'].value_counts())
display(student_not_assessed_df[['date_registration', 'date_unregistration']].describe())

print("""
The Students that have been assessed.
""")

students_assessed_df = student_df[student_df['id_student'].isin(students_assessed)]
display(students_assessed_df.nunique())
display(students_assessed_df['final_result'].value_counts())
display(students_assessed_df[['date_registration', 'date_unregistration']].describe())

print("""
The Students that have been not been assessed but interacted.
""")

student_not_assessed_df = vle_df[~vle_df['id_student'].isin(students_assessed)].copy()
display(student_not_assessed_df.nunique())


In [ ]:
#df = df.astype({'gender':'string', 'imd_band':'string', 'highest_education':'string', 'age_band':'string', 'region':'string', 'final_result':'string'})
#df.dtypes


In [ ]:
display(df.nunique())
#display(df.value_counts("gender"))
#display(df.value_counts("imd_band"))
#display(df.value_counts("highest_education"))
#print(f"A-Level is equivilent to high school cert, HE Qualification is level 1 ro 2 of University, ")
#display(df.value_counts("age_band"))
#display(df.value_counts("num_of_prev_attempts"))
#display(df.value_counts("region"))
display(df.value_counts("final_result"))

In [ ]:
# distinction is a higher form of pass
# withdrawn would be a fail if the student would not complete the course both fail and withdrawn is a form of failure.
# Possible to remove withdrawn
df_edited = df.copy()
df_edited["final_result"] = df_edited["final_result"].replace({'Distinction': 'Pass', 'Withdrawn': 'Fail'})
df_edited["final_result"] = df_edited["final_result"].replace({'Pass': '1', 'Fail': '0'})
df_edited = df_edited.astype({'day': 'float32', 'clicks': 'float32', 'num_of_prev_attempts': 'float32', 'final_result': 'float32'})
display(df_edited["final_result"].unique())
display(df_edited.dtypes)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [ ]:
df_edited = pd.get_dummies(df_edited)
df_edited.head()

In [ ]:
# combine pass and distinction
# combine widthdrawn and fail
X = df_edited.drop('final_result', axis=1).values
y = df_edited['final_result'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)


In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

X_train_scaled.shape

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])

# X shape / input feature / columns rule of thumbs is normally 2-3 times the ammount so 43 * 2 or 3 is 86-129 so i will try choosing 90 for the first hidden node
hidden_nodes1 = 90
hidden_nodes2 = 60
hidden_nodes3 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes1, input_shape=(number_input_features,), activation='relu'))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes2, activation='relu'))
# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes3, activation='relu'))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train, epochs=20)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
nn.save("models/Student_Pass1Fail_relu90+relu60+relu30+sigmoid.h5")

In [ ]:
# plotting the accuracy
df_edited = pd.DataFrame(fit_model.history, index = range(1, len(fit_model.history['loss'])+1))
df_edited.plot(y = 'accuracy')